# AmebaブログのヘルプページFAQデータセットの作成

## 概要

**データセットを以下の2種類のデータとして再構築する**  

`df_qa`: Queryとその回答に必要な参考情報のID
- Query: 質問
- ID: 回答に必要な参考情報のID

`df_ref`:
- ID: 回答に必要な参考情報のID
Reference: 参考情報

## imports

In [4]:
import os
import copy
import pandas as pd

import warnings
warnings.simplefilter('ignore')

## 1. [ai-shift/ameba_faq_search](https://huggingface.co/datasets/ai-shift/ameba_faq_search) からデータをダウンロード

## 2. データの読み込み

In [5]:
from datasets import load_dataset

# LLMで生成したユーザーの想定質問
query_dset = load_dataset("ai-shift/ameba_faq_search")

# 検索対象のFAQデータ
faq_dset = load_dataset("ai-shift/ameba_faq_search", data_files={"faq": "target_faq.csv"})

Generating train split:   0%|          | 0/1313 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/792 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/837 [00:00<?, ? examples/s]

Generating faq split: 0 examples [00:00, ? examples/s]

In [6]:
query_dset.keys()

dict_keys(['train', 'validation', 'test'])

In [7]:
faq_dset.keys()

dict_keys(['faq'])

## 3. 再構築

In [8]:
# FAQ
df_ref = pd.DataFrame(faq_dset["faq"])

# Questions
df_train = pd.DataFrame(query_dset["train"])
df_val   = pd.DataFrame(query_dset["validation"])
df_test  = pd.DataFrame(query_dset["test"])

In [9]:
df_ref.head()

,ID,Title,Content
0,faq_000,画面が表示されない・記事が投稿できない,Amebaで障害が発生している場合は、スタッフブログやX(旧Twitter)でお知らせします...
1,faq_001,ログインできません,Amebaアプリにてログインができない場合、一度スマートフォンブラウザにて以下ログインフォー...
2,faq_002,退会したいのにログインができない,退会手続きはご自身でログインし行う必要があります。アメーバID・パスワードが分からない場合や...
3,faq_003,退会方法について,退会にあたってログインができない場合は、ヘルプページ(ログインできない)をご確認ください。\...
4,faq_004,お知らせメールの設定場所・配信停止設定がわかりません,以下ページで設定いただいているメールアドレス宛に送信されます。\n■会員登録情報ページ内「メ...


In [10]:
df_train.head()

,ID,Query,difficulty
0,faq_001,パスワードを忘れた,easy
1,faq_001,メールアドレスにパスワード再設定通知が届かない,easy
2,faq_001,登録メールアドレスが分からない,easy
3,faq_001,パスワード再設定をしてもログインできない,easy
4,faq_001,ブログ管理画面へログインできない,easy


<font size=4 color="indianred">`df_train`のQueryに対する回答に必要な情報は、IDで紐づいている`df_faq`のContentになる。</font>

In [11]:
df_train["dataset"] = "train"
df_test["dataset"]  = "test"
df_val["dataset"]   = "val"

In [12]:
df = pd.concat([
    df_train,df_val,df_test
],axis=0)

In [13]:
df.shape

(2942, 4)

In [14]:
df_qa = df
df_qa.head(2)

,ID,Query,difficulty,dataset
0,faq_001,パスワードを忘れた,easy,train
1,faq_001,メールアドレスにパスワード再設定通知が届かない,easy,train


In [15]:
df_ref = df_ref.rename(columns={"Content":"Reference"}).drop("Title",axis=1)

In [16]:
df_ref.head(2)

,ID,Reference
0,faq_000,Amebaで障害が発生している場合は、スタッフブログやX(旧Twitter)でお知らせします...
1,faq_001,Amebaアプリにてログインができない場合、一度スマートフォンブラウザにて以下ログインフォー...


## 4. 保存

In [18]:
dataset_dir = "./dataset/01_Abema"
os.makedirs(dataset_dir)
df_ref.to_csv("./dataset/01_Abema/Reference.csv")
df_qa.to_csv("./dataset/01_Abema/QA.csv")

In [19]:
!pwd

/Users/ryoyakatafuchi/Desktop/workspace/vector/notebooks


In [90]:
df_ref.shape,df_ref["ID"].nunique()

((461, 2), 461)

## Abema FAQを利用している参考記事
- https://www.ai-shift.co.jp/techblog/3710
- https://www.ai-shift.co.jp/techblog/3761